In [ ]:
from tweepy import OAuthHandler #auth based on ID in other file
from tweepy import API #Twitter API
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from textblob import TextBlob
from scipy.stats import gaussian_kde
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

In [ ]:
class AuthenticateTwitter():
    #API Set up
    def AuthenticTwitterApp(self):
        Authenticate = OAuthHandler("CONSUMER TOKEN","CONSUMER S TOKEN")
        Authenticate.set_access_token("ACCESS TOKEN","ACCESS S TOKEN")
        
        return Authenticate

In [ ]:
class ClientTwitter():
    #API Access
    def __init__ (self, user_twitter=None):
        self.Authenticate = AuthenticateTwitter().AuthenticTwitterApp()
        self.twitter_client = API(self.Authenticate) 
        self.user_twitter = user_twitter
        
    def Twitter_API(self):
        
        return self.twitter_client

In [ ]:
class AnalyseTweetTwitter():
        
    def D_F_Tweets(self, TTweets):
        #DATA CLEANSING
        #SHOWS TWEET
        dftweet = pd.DataFrame(data=[tweet.text for tweet in TTweets], columns=['Tweets']) 
        #SHOWS SENTIMENT OF TWEET        
        dftweet['TextBlob SA'] = np.array([SentimentTwitter().TextBlob_Tweet_Analyse_Sent(tweet) for tweet in dftweet['Tweets']])
        dftweet['Vader SA'] = np.array([SentimentTwitter().Vader_Tweet_Analyse_sent(tweet) for tweet in dftweet['Tweets']])
        #SHOWS HOW MANY LIKES
        dftweet['Likes']= np.array([tweet.favorite_count for tweet in TTweets])
        #WHEN CREATED
        dftweet['Created']= np.array([tweet.created_at for tweet in TTweets])
        #SHOWS SOURCE
        dftweet['Source of tweet']= np.array([tweet.source for tweet in TTweets])
        #SHOWS HOW MANY RETWEETS
        dftweet['Retweets']= np.array([tweet.retweet_count for tweet in TTweets])
        
        return dftweet 

In [ ]:
class SentimentTwitter():
    
    #Cleansing of the tweet
    def Tweet_Cleansing(self, tweet):
        #takes out the characters that dont need to be there
        return ' '.join(re.sub("(\w+:\/\/\S+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet).split())
    
    def TextBlob_Tweet_Analyse_Sent(self, tweet):
        TextBlobAnalysedTweet = TextBlob(self.Tweet_Cleansing(tweet))#textblob gets the clean tweet and analyse's the tweet
        #returns a rounded up number to analyse better
        if TextBlobAnalysedTweet.sentiment.polarity >= 0.5:
            return 1
        elif TextBlobAnalysedTweet.sentiment.polarity > 0 and TextBlobAnalysedTweet.sentiment.polarity < 0.5:
            return 0.5
        elif TextBlobAnalysedTweet.sentiment.polarity == 0:
            return 0
        elif TextBlobAnalysedTweet.sentiment.polarity < 0 and TextBlobAnalysedTweet.sentiment.polarity > -0.5:
            return -0.5
        else:
            return -1
    
    def Vader_Tweet_Analyse_sent(self, tweet):
        VaderAnalysedTweet = SIA()
        vs = VaderAnalysedTweet.polarity_scores(self.Tweet_Cleansing(tweet))
        if vs['compound'] >= 0.5:
            return 1
        elif vs['compound'] > 0 and vs['compound'] < 0.5:
            return 0.5
        elif vs['compound'] == 0:
            return 0
        elif vs['compound'] < 0 and vs['compound'] > -0.5:
            return -0.5
        else:
            return -1
    

In [ ]:
if __name__ == "__main__":
    twitter_client = ClientTwitter()
    analyse_tweet = AnalyseTweetTwitter()
    api = twitter_client.Twitter_API()
    
    TTweets = api.user_timeline(screen_name = "Nigel_Farage", count = 50)
    Tweets = api.user_timeline(screen_name = "BearGrylls", count = 50)
    
    #TTweets = api.search(q = "cats", lang = "en", result_type = "recent", count = 20)
    
    
    dftweet = analyse_tweet.D_F_Tweets(TTweets) #gets what you want out of the tweet
    dftweet2 = analyse_tweet.D_F_Tweets(Tweets)
    print(dftweet)
    print(dftweet2)

In [ ]:
rng = dftweet['TextBlob SA']
x = dftweet['Created']
y = dftweet['Likes']
colors = rng 
sizes = 100 + (rng * 20)  

rng2 = dftweet2['TextBlob SA']
x2 = dftweet2['Created']
y2 = dftweet2['Likes']
colors2 = rng2
sizes2 = 100 + (rng2 * 20)


plt.scatter(x, y, c=colors, s=sizes, alpha=0.3,
            cmap='viridis')
plt.colorbar();
plt.scatter(x2, y2, c=colors2, s=sizes2, alpha=0.3,
            cmap='Set3')
plt.colorbar();  # show color scale

In [ ]:
SAToTime = pd.Series(data = dftweet['TextBlob SA'].values, index = dftweet['Created'])
SAToTime.plot(figsize=(9,3), color='b')
SAToTime = pd.Series(data = dftweet['Vader SA'].values, index = dftweet['Created'])
SAToTime.plot(figsize=(9,3), color='r')
plt.title('Difference in SA between the programs')
plt.show()

In [ ]:
plt.style.use('seaborn-dark-palette')
data = dftweet2['Vader SA'].values
data2 = dftweet2['TextBlob SA'].values

bins = np.linspace(-1,1)

plt.hist(data,bins, alpha = 0.3, label=['Vader', 'y'])
plt.hist(data2,bins, alpha = 0.3, label=['TextBlog','y'])
plt.legend(loc='upper right')
plt.title('Difference between the two SA programs')
plt.show()

In [ ]:
plt.style.use('seaborn-dark-palette')
data = dftweet['Vader SA'].values
data2 = dftweet['TextBlob SA'].values

bins = np.linspace(-1,1)

plt.hist(data,bins, alpha = 0.3, label=['Vader', 'y'])
plt.hist(data2,bins, alpha = 0.3, label=['TextBlog','y'])
plt.legend(loc='upper right')
plt.title('Difference between the two SA programs')
plt.show()

In [ ]:
unique_elements, counts_elements = np.unique(dftweet['Vader SA'].values, return_counts= True)
labels =['negative', 'marginally negative','neutral','marginally positive','positive'] 
plt.pie(counts_elements, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90 )
plt.axis('equal')
plt.show()

In [ ]:
unique_elements, counts_elements = np.unique(dftweet2['TextBlob SA'].values, return_counts= True)
print(counts_elements)
labels =['negative', 'marginally negative','neutral','marginally positive','positive']  
plt.pie(counts_elements, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90 )
plt.axis('equal')
plt.show()